# Data Curation

This python notebook is used to extract and process a dataset from the wikipedia, used to train YAPPER.

The flow is as follows:

* Extract English-written Wiki pages in science, geography, ecnonomics, and history
* Preproccess extracted text to remove artifacts and whitespace
* Semantically chunk them to ~ 500 words long chunks of text. Drop all chunks less than 400 words
* Generate metadata for each chunk
* Ensure there are no near duplicates through embedding similarity
* Split to train/eval/test
* Generate dataset summary

## Imports & Initializations

In [2]:
import requests
import mwparserfromhell
import re
import time
import html
import json
import uuid
import numpy as np
import pandas as pd
import duckdb as ddb
from matplotlib import pyplot as plt
from dotenv import dotenv_values
from tqdm import tqdm
import hashlib
from collections import deque
import asyncio
import inspect
import os

import semchunk
from google import genai

In [3]:
# Get Env vars
config = dotenv_values(".env")
config["WIKI_API"] = "https://en.wikipedia.org/w/api.php"
config["WIKI_BATCH_SIZE"] = 50
config["WIKI_HEADERS"] = {"User-Agent": "WikiDatasetBot/0.1 (contact: your_email@example.com)"}

In [4]:
# Duckdb connection
con = ddb.connect("/hpc/home/bfa6/work/data/yapper/database.duckdb")

In [5]:
# Gemini client
client = genai.Client(api_key=config["GEMINI_API_KEY"])

In [6]:
# Semchunk chunker
chunker = semchunk.chunkerify(lambda text: len(text.split()), 500)

## Functions

### Wiki related functions

In [7]:
# 1) utility: ensure category prefix
def ensure_cat_prefix(cat_name: str) -> str:
    return cat_name if cat_name.startswith("Category:") else "Category:" + cat_name

# 2) generator: category members (pages / subcats / files) - top-level generator function
def category_members_generator(cat_title: str, cmtype: str = "page", cmnamespace: int = 0, limit: int = 500, delay: float = 0.0):
    """
    Yield category members for a category (page dicts from API).
    cmtype: "page" | "subcat" | "file"
    cmnamespace: namespace filter (0 for articles, 14 for categories)
    """
    params = {
        "action": "query",
        "format": "json",
        "list": "categorymembers",
        "cmtitle": cat_title,
        "cmtype": cmtype,
        "cmnamespace": cmnamespace,
        "cmlimit": str(limit),
        "formatversion": 2
    }
    cont = {}
    while True:
        p = params.copy()
        p.update(cont)
        r = requests.get(config["WIKI_API"], params=p, headers=config["WIKI_HEADERS"], timeout=30)
        r.raise_for_status()
        j = r.json()
        members = j.get("query", {}).get("categorymembers", [])
        for m in members:
            yield m
        if "continue" in j:
            cont = j["continue"]
        else:
            break
        if delay:
            time.sleep(delay)

# 3) gather titles from one seed category (BFS over subcats optional)
def gather_titles_from_category(seed_category: str, recurse: bool = False, max_depth: int = 1, max_pages_per_category: int = 1000, delay: float = 0.0):
    """
    Return a set of page titles (namespace 0) under seed_category.
    """
    seed_cat = ensure_cat_prefix(seed_category)
    queue = deque()
    queue.append((seed_cat, 0))
    seen_cats = {seed_cat}
    titles = set()

    while queue:
        cat, depth = queue.popleft()
        count = 0
        for m in category_members_generator(cat, cmtype="page", cmnamespace=0, limit=500, delay=delay):
            title = m.get("title")
            if title:
                titles.add(title)
            count += 1
            if count >= max_pages_per_category:
                break

        if not recurse or depth >= max_depth:
            continue

        for sub in category_members_generator(cat, cmtype="subcat", cmnamespace=14, limit=500, delay=delay):
            sub_title = sub.get("title")
            if sub_title and sub_title not in seen_cats:
                seen_cats.add(sub_title)
                queue.append((sub_title, depth + 1))

    return titles

# 4) fetch metadata for a list of pageids (batched)
def fetch_pages_metadata_by_ids(pageids, delay: float = 0.0):
    """
    Return list of metadata dicts for the given pageids.
    Each dict contains pageid, title, ns, fullurl, length, lastrevid, categories (list).
    """
    results = []
    for i in tqdm(range(0, len(pageids), config["WIKI_BATCH_SIZE"]), desc="Fetching metadata", leave=False):
        chunk = pageids[i:i+config["WIKI_BATCH_SIZE"]]
        params = {
            "action": "query",
            "format": "json",
            "pageids": "|".join(str(x) for x in chunk),
            "prop": "info|categories",
            "inprop": "url",
            "cllimit": "max",
            "formatversion": 2
        }
        r = requests.get(config["WIKI_API"], params=params, headers=config["WIKI_HEADERS"], timeout=30)
        r.raise_for_status()
        j = r.json()
        pages = j.get("query", {}).get("pages", [])
        for p in pages:
            rec = {
                "pageid": p.get("pageid"),
                "title": p.get("title"),
                "ns": p.get("ns"),
                "fullurl": p.get("fullurl"),
                "length": p.get("length"),
                "lastrevid": p.get("lastrevid"),
                "categories": [c.get("title") for c in (p.get("categories") or [])]
            }
            results.append(rec)
        if delay:
            time.sleep(delay)
    return results

# 5) main orchestrator (no nested functions)
def get_pages_from_categories(seed_categories, recurse: bool = False, max_depth: int = 1, max_pages_per_seed: int = 1000, delay_between_api_calls: float = 0.0):
    """
    Given a list of seed category names, return a list of enriched page metadata dicts.
    - seed_categories: list of strings (with or w/o "Category:" prefix)
    - returns list of dicts with page metadata and seed_category field.
    """
    # map pageid -> seed_category (first seed encountered)
    pageid_to_seed = {}
    for seed in tqdm(seed_categories, desc="Seed categories"):
        seed_pref = ensure_cat_prefix(seed)
        titles = gather_titles_from_category(seed_pref, recurse=recurse, max_depth=max_depth, max_pages_per_category=max_pages_per_seed, delay=delay_between_api_calls)
        # gather pageids for these titles using a query (batch)
        # We need to convert titles -> pageids in batches
        titles_list = list(titles)
        for i in tqdm(range(0, len(titles_list), config["WIKI_BATCH_SIZE"]), desc=f"Converting titles for {seed_pref}", leave=False):
            chunk = titles_list[i:i+config["WIKI_BATCH_SIZE"]]
            params = {
                "action": "query",
                "format": "json",
                "titles": "|".join(chunk),
                "prop": "info",
                "inprop": "url",
                "formatversion": 2
            }
            r = requests.get(config["WIKI_API"], params=params, headers=config["WIKI_HEADERS"], timeout=30)
            r.raise_for_status()
            j = r.json()
            pages = j.get("query", {}).get("pages", [])
            for p in pages:
                pid = p.get("pageid")
                if pid and pid not in pageid_to_seed:
                    pageid_to_seed[pid] = {"seed_category": seed_pref}
            if delay_between_api_calls:
                time.sleep(delay_between_api_calls)

    if not pageid_to_seed:
        return []

    # fetch metadata for collected pageids
    pageids = list(pageid_to_seed.keys())
    meta_list = fetch_pages_metadata_by_ids(pageids, delay=delay_between_api_calls)

    # attach seed_category
    for m in meta_list:
        pid = m.get("pageid")
        m["seed_category"] = pageid_to_seed.get(pid, {}).get("seed_category")

    # sort deterministically
    meta_list.sort(key=lambda x: (x.get("seed_category") or "", x.get("title") or ""))
    return meta_list


In [8]:
def _clean_wikitext(wikitext: str) -> str:
    """
    Convert wikitext to plain text paragraphs only.
    """
    
    # Remove comments
    text = re.sub(r'<!--.*?-->', '', wikitext, flags=re.DOTALL)
    
    # Remove all references (including nested ones)
    while '<ref' in text:
        text = re.sub(r'<ref[^>]*>.*?</ref>', '', text, flags=re.DOTALL)
        text = re.sub(r'<ref[^>]*\/>', '', text)
    
    # Remove infoboxes and templates aggressively
    def remove_nested_braces(s):
        while '{{' in s:
            start = s.find('{{')
            if start == -1:
                break
            count = 0
            i = start
            while i < len(s):
                if s[i:i+2] == '{{':
                    count += 1
                    i += 2
                elif s[i:i+2] == '}}':
                    count -= 1
                    i += 2
                    if count == 0:
                        s = s[:start] + s[i:]
                        break
                else:
                    i += 1
            else:
                break
        return s
    
    text = remove_nested_braces(text)
    
    # Remove tables
    text = re.sub(r'\{\|.*?\|\}', '', text, flags=re.DOTALL)
    
    # Remove files/images with nested brackets
    def remove_files(s):
        while True:
            match = re.search(r'\[\[(?:File|Image):', s, re.IGNORECASE)
            if not match:
                break
            start = match.start()
            count = 0
            i = start
            while i < len(s):
                if s[i:i+2] == '[[':
                    count += 1
                    i += 2
                elif s[i:i+2] == ']]':
                    count -= 1
                    i += 2
                    if count == 0:
                        s = s[:start] + s[i:]
                        break
                else:
                    i += 1
            else:
                break
        return s
    
    text = remove_files(text)
    
    # Remove categories
    text = re.sub(r'\[\[Category:.*?\]\]', '', text, flags=re.IGNORECASE)
    
    # Convert wikilinks: [[Link|Display]] -> Display, [[Link]] -> Link
    text = re.sub(r'\[\[(?:[^|\]]*\|)?([^\]]+)\]\]', r'\1', text)
    
    # Remove external links
    text = re.sub(r'\[https?://[^\]]+\]', '', text)
    
    # Remove section headers
    text = re.sub(r'={2,}.*?={2,}', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove formatting
    text = re.sub(r"'{2,}", '', text)
    
    # Filter lines
    lines = []
    skip_sections = {'Footnotes', 'Bibliography', 'References', 'External links', 'See also', 'Notes', 'Further reading'}
    
    for line in text.split('\n'):
        line = line.strip()
        if not line or line in skip_sections:
            continue
        if re.match(r'^[\*:;\|!{}\[\]]', line):
            continue
        lines.append(line)
    
    return '\n\n'.join(lines)


def get_wikipedia_page_text(title: str, sleep: float = 0.5) -> str:
    """
    Fetch the plain text of a Wikipedia page (English) given its title.

    Args:
        title (str): The Wikipedia page title (e.g., "Artificial intelligence").
        sleep (float): Optional pause between requests (in seconds) to be polite.

    Returns:
        str: Cleaned plain-text content of the page, or empty string if not found.
    """
    params = {
        "action": "query",
        "format": "json",
        "prop": "revisions",
        "rvprop": "content",
        "titles": title,
        "formatversion": 2,
        "redirects": 1
    }

    try:
        r = requests.get(
            config["WIKI_API"],
            params=params,
            headers=config["WIKI_HEADERS"],
            timeout=10
        )
        r.raise_for_status()
        data = r.json()
        pages = data.get("query", {}).get("pages", [])
        if not pages or "revisions" not in pages[0]:
            return ""

        # Raw wikitext
        wikitext = pages[0]["revisions"][0]["content"]
        text = _clean_wikitext(wikitext)

        time.sleep(sleep)
        return text.strip()

    except Exception as e:
        print(f"[Error] {title}: {e}")
        return ""


### Duckdb Functions

In [9]:
# Create dataset table
def create_tables() -> bool:
    ddl = """
    CREATE OR REPLACE TABLE wiki_pages (
        id UUID DEFAULT uuid() PRIMARY KEY,
        wiki_page_id INTEGER UNIQUE,
        title TEXT,
        url TEXT,
        categories JSON  -- store list of strings as JSON
    );

    CREATE OR REPLACE TABLE wiki_chunks (
        id UUID DEFAULT uuid() PRIMARY KEY,
        chunk TEXT,
        questions JSON,  -- store list of strings as JSON
        hash TEXT UNIQUE,
        page_id UUID 
    );
    """

    try:
        con.execute(ddl)
    except Exception as e:
        print(f"Failed to create tables: {e}")
        return False

    return True

In [10]:
def insert_wiki_pages(pages: list[dict]) -> bool:
    if not pages:
        return True

    try:
        data = [
            (p["wiki_page_id"], p["title"], p["url"], json.dumps(p.get("categories", [])))
            for p in pages
        ]
        con.executemany(
            "INSERT INTO wiki_pages (wiki_page_id, title, url, categories) VALUES (?, ?, ?, ?)",
            data
        )
        return True
    except Exception as e:
        print(f"Batch insert into wiki_pages failed: {e}")
        return False


def insert_wiki_chunks(chunks: list[dict]) -> bool:
    if not chunks:
        return True

    try:
        data = [
            (c["chunk"], json.dumps(c.get("questions", [])), c["hash"], c["page_id"])
            for c in chunks
        ]
        con.executemany(
            "INSERT INTO wiki_chunks (chunk, questions, hash, page_id) VALUES (?, ?, ?, ?)",
            data
        )
        return True
    except Exception as e:
        print(f"Batch insert into wiki_chunks failed: {e}")
        return False


In [11]:
def retrieve_wiki_pages():
    query = "SELECT * from wiki_pages"

    df_result = con.execute(query).df()

    return df_result.to_dict('records')

In [21]:
def generate_data_stats():
    query = """
    SELECT
        (SELECT COUNT(DISTINCT id) FROM wiki_pages) AS num_pages,
        (SELECT COUNT(DISTINCT id) FROM wiki_chunks) AS num_chunks,

        -- Chunk word counts
        (SELECT MIN(array_length(regexp_split_to_array(trim(chunk), '\\s+'), 1))
         FROM wiki_chunks WHERE chunk IS NOT NULL AND trim(chunk) <> '') AS min_chunk_length,

        (SELECT MAX(array_length(regexp_split_to_array(trim(chunk), '\\s+'), 1))
         FROM wiki_chunks WHERE chunk IS NOT NULL AND trim(chunk) <> '') AS max_chunk_length,

        (SELECT AVG(array_length(regexp_split_to_array(trim(chunk), '\\s+'), 1)::numeric)
         FROM wiki_chunks WHERE chunk IS NOT NULL AND trim(chunk) <> '') AS avg_chunk_length,

        (SELECT percentile_disc(0.5)  WITHIN GROUP (ORDER BY array_length(regexp_split_to_array(trim(chunk), '\\s+'), 1))
         FROM wiki_chunks WHERE chunk IS NOT NULL AND trim(chunk) <> '') AS median_chunk_length,

        (SELECT percentile_disc(0.25) WITHIN GROUP (ORDER BY array_length(regexp_split_to_array(trim(chunk), '\\s+'), 1))
         FROM wiki_chunks WHERE chunk IS NOT NULL AND trim(chunk) <> '') AS q1_chunk_length,

        (SELECT percentile_disc(0.75) WITHIN GROUP (ORDER BY array_length(regexp_split_to_array(trim(chunk), '\\s+'), 1))
         FROM wiki_chunks WHERE chunk IS NOT NULL AND trim(chunk) <> '') AS q3_chunk_length,

    ;
    """
    df_result = con.execute(query).df()
    
    return df_result.to_dict('records')


In [13]:
def save_tables(dir_path: str):
    os.makedirs(dir_path, exist_ok=True)

    con.execute("SELECT * FROM wiki_pages").df().to_csv(os.path.join(dir_path, "wiki_pages.csv"), index=False)
    con.execute("SELECT * FROM wiki_chunks").df().to_csv(os.path.join(dir_path, "wiki_chunks.csv"), index=False)

### Gemini Question generation Functions

In [14]:
def ask_gemini(
    contents: str,
    model: str = "gemini-2.5-flash",
    retries: int = 3,
    backoff_factor: float = 2.0,
) -> str:
    """
    Generate text using Gemini with retry and standard exception handling.
    """
    for attempt in range(1, retries + 1):
        try:
            response = client.models.generate_content(
                model=model,
                contents=contents,
            )
            return response.text.strip()

        except (ConnectionError, TimeoutError) as e:
            print(f"Network issue on attempt {attempt}: {e}")
        except Exception as e:
            print(f"Error on attempt {attempt}: {e}")

        # Retry if not the last attempt
        if attempt < retries:
            sleep_time = backoff_factor ** (attempt - 1)
            print(f"Retrying in {sleep_time:.1f} seconds...")
            time.sleep(sleep_time)
        else:
            print("All retries failed.")
            return f"Error: {e}"

    return "Failed to generate response after multiple attempts."

In [15]:
def generate_questions(
    context: str,
    model: str = "gemini-2.5-flash",
    retries: int = 3,
    backoff_factor: float = 2.0,
) -> dict:
    """
    Generate Q&A pairs from a given context using Gemini,
    returning a JSON object like:
      {"QAs": [{"Question": "...", "Answer": "..."}]}
    """
    system_prompt = (
        "You are an expert question generator. "
        "Given the following text, generate several question–answer pairs "
        "that test understanding of its key ideas. "
        "Answers must be short and concise — ideally one short phrase or sentence, not long explanations. "
        "Respond ONLY in valid JSON with this structure:\n"
        '{"QAs": [{"Question": "<string>", "Answer": "<string>"}]}'
    )

    for attempt in range(1, retries + 1):
        try:
            response = client.models.generate_content(
                model=model,
                contents=[system_prompt, context],
                config={"response_mime_type": "application/json"},
            )

            # If response is valid JSON, .parsed gives a Python dict
            # print(response.text)
            return json.loads(response.text)

        except (ConnectionError, TimeoutError) as e:
            print(f"Network issue on attempt {attempt}: {e}")
        except Exception as e:
            print(f"Error on attempt {attempt}: {e}")

        if attempt < retries:
            sleep_time = backoff_factor ** (attempt - 1)
            print(f"Retrying in {sleep_time:.1f} seconds...")
            time.sleep(sleep_time)
        else:
            print("All retries failed.")
            return {"QAs": []}

    return {"QAs": []}

### Semantic Chunking Functions

In [16]:
def get_chunks(content: str, threshold: int = 400) -> list[str]:
    chunks = chunker(content)
    valid_chunks = [chunk for chunk in chunks if len(chunk.split()) > threshold]
    
    return valid_chunks

### Miscellaneous functions

In [17]:
def hash_text(text: str):
    text_bytes = text.encode('utf-8')
    hash_object = hashlib.sha256(text_bytes)
    hash_hex = hash_object.hexdigest()
    
    return hash_hex


### Final Functions

In [18]:
def fetch_all_articles(
    seed_categories: list, 
    recurse: bool = False, 
    max_depth: int = 0, 
    max_pages_per_seed: int = 10000, 
    delay_between_api_calls: float = 0.01
):
    # Get articles
    pages = get_pages_from_categories(
        seed_categories=seed_categories,
        recurse=recurse,
        max_depth=max_depth, 
        max_pages_per_seed=max_pages_per_seed, 
        delay_between_api_calls=delay_between_api_calls
    )

    # Save to wiki_pages
    wiki_pages = [{"wiki_page_id": page["pageid"], "title": page["title"], "url": page["fullurl"], "categories": page["categories"]} for page in pages]
    result = insert_wiki_pages(pages=wiki_pages)


    return result


In [25]:
async def generate_wiki_chunks(model: str = "gemini-2.5-flash", concurrency: int = 8, chunk_min_threshold: int = 400):
    # Get list of wiki page titles and ids
    pages = retrieve_wiki_pages()

    records = []
    sem = asyncio.Semaphore(concurrency)

    # Loop (async) to generate all the chunks
    for page in tqdm(pages, desc="Generating chunks"):
        # Get content
        content = get_wikipedia_page_text(page["title"])

        # get chunks
        chunks = get_chunks(content, threshold=chunk_min_threshold)

        # for each chunk, generate Q&As then store the chunk into the database
        async def process_chunk(chunk):
            async with sem:
                qa = generate_questions(context=chunk, model=model)
                return {
                    "chunk": chunk,
                    "questions": qa["QAs"],
                    "hash": hash_text(chunk),
                    "page_id": page["id"]
                }

        tasks = [asyncio.create_task(process_chunk(chunk)) for chunk in chunks]
        results = await asyncio.gather(*tasks)
        records.extend(results)

    # Store all chunks into the database
    insert_wiki_chunks(records)

    return True

In [20]:
async def generate_dataset(
    seed_categories: list, 
    recurse: bool = False, 
    max_depth: int = 0, 
    max_pages_per_seed: int = 10000, 
    delay_between_api_calls: float = 0.01,
    model: str = "gemini-2.5-flash",
    refresh_tables: bool = True,
    page_processing_concurrency: int = 8,
    chunk_min_threshold: int = 400
):
    # Clear tables if refresh_tables=True
    if refresh_tables:
        flag = create_tables()

        if not flag:
            raise ValueError("Failed to create tables")

    # Find the articles
    flag = fetch_all_articles(
        seed_categories=seed_categories,
        recurse=recurse,
        max_depth=max_depth, 
        max_pages_per_seed=max_pages_per_seed, 
        delay_between_api_calls=delay_between_api_calls
    )

    if not flag:
        raise ValueError("Failed to fetch articles")

    # For each article: Chunk, then save into the chunk table
    flag = await generate_wiki_chunks(model=model, concurrency=page_processing_concurrency, chunk_min_threshold=chunk_min_threshold)

    if not flag:
        raise ValueError("Failed to generate wiki chunks")

    # Print some dataset statistics
    stats = generate_data_stats()[0]

    print("Number of pages:", stats['num_pages'])
    print("Number of chunks:", stats['num_chunks'])
    print("Min chunk length (words):", stats['min_chunk_length'])
    print("Max chunk length (words):", stats['max_chunk_length'])
    print("Avg chunk length (words):", stats['avg_chunk_length'])
    print("Median chunk length (words):", stats['median_chunk_length'])
    print("Q1 (25th percentile):", stats['q1_chunk_length'])
    print("Q3 (75th percentile):", stats['q3_chunk_length'])
    print("Total questions:", stats['num_questions'])
    print("Avg questions per chunk:", stats['avg_num_questions_per_chunk'])

    return True

## Testing Grounds

In [26]:
seeds = ["Science", "Geography", "Economics", "History"]
await generate_dataset(
    seed_categories=seeds, 
    recurse=False, 
    max_depth=1, 
    max_pages_per_seed=1000, 
    delay_between_api_calls=0.01,
    model="gemini-2.5-flash",
    refresh_tables=True,
    page_processing_concurrency=10
)

Generating chunks: 100%|██████████| 51/51 [12:24<00:00, 14.61s/it]


Number of pages: 51
Number of chunks: 102
Min chunk length (words): 401
Max chunk length (words): 500
Avg chunk length (words): 459.47058823529414
Median chunk length (words): 462
Q1 (25th percentile): 441
Q3 (75th percentile): 478
Total questions: 0.0
Avg questions per chunk: 0.0


True

In [22]:
save_tables(dir_path="/hpc/home/bfa6/work/data/yapper")

In [24]:
con.execute("SELECT * FROM wiki_chunks").df()

,id,chunk,questions,hash,page_id
0,bcaf0b72-4500-4348-9042-63765ca52645,An economic impact analysis (EIA) examines the...,"{""QAs"": [{""Question"": ""What does an Economic I...",80359810820c67b89474865165b1ce38b68c6df0a374ec...,7b736f4f-89cf-43bf-82e0-8337d45a22b3
1,b406cdc2-6c50-4dc6-a5a5-380f2bcd369c,Economics () is a social science that studies ...,"{""QAs"": [{""Question"": ""What is the primary foc...",e09ee23dd8215fa7f0edb1e75550b6c9049d99ebd53bb7...,c7a27ee4-c1aa-462c-9d84-0cf636a62d4c
2,fea860ac-11da-4b66-b1b5-64e6f9f8cb59,Questions regarding distribution of resources ...,"{""QAs"": [{""Question"": ""Who did some economic h...",c6254cd1bcd5f273dc394b6d0927109c49fa8aa8d55735...,c7a27ee4-c1aa-462c-9d84-0cf636a62d4c
3,7878af4e-2ac4-4546-b83f-d4441a2d00a4,Smith discusses potential benefits of speciali...,"{""QAs"": [{""Question"": ""What did Adam Smith ide...",c84b6660dc8ebe73f2e6612fb8f30f1153f1b1914b41ce...,c7a27ee4-c1aa-462c-9d84-0cf636a62d4c
4,00b87c29-76a3-47b9-9911-24989d2f568c,Value theory was important in classical theory...,"{""QAs"": [{""Question"": ""What did Adam Smith ide...",3c2a1e21cda9bf54abb8c98e430639e624173b0aae4806...,c7a27ee4-c1aa-462c-9d84-0cf636a62d4c
...,...,...,...,...,...
97,d750147b-2785-4aba-9da6-7d495a2d8001,There are different schools of thought in the ...,"{""QAs"": [{""Question"": ""What is the most popula...",7db5488084968b152e0e5b6ebb30ea2ee9e8b55619a203...,3a497910-e7b9-4f2b-987f-1b81d54a143c
98,af7d0451-c5ea-4d1c-a23f-f8643bb3a43f,The scientific community is a network of inter...,"{""QAs"": [{""Question"": ""What is the primary pur...",352088935319a776400233161efb6af9cd346ed061572f...,3a497910-e7b9-4f2b-987f-1b81d54a143c
99,8e82fac4-bcf3-49a3-bd55-bed7c9d7c5e0,Funding of science is often through a competit...,"{""QAs"": [{""Question"": ""What is the typical pro...",c44824973302bf75f6a10974c50ff3c08cc98253766747...,3a497910-e7b9-4f2b-987f-1b81d54a143c
100,cf353e53-7fd2-4272-89ff-0d07875b2651,"Science magazines such as New Scientist, Scien...","{""QAs"": [{""Question"": ""Name two types of media...",b107096835f5143d2670af76243ca5f33bdedfb629ac57...,3a497910-e7b9-4f2b-987f-1b81d54a143c
